In [99]:
import tensorflow as tf
import pandas as pd
import numpy as np
import json
import glob
from sklearn.model_selection import train_test_split

In [4]:
# Base path to folder containing data samples.
BASE_PATH = '/Users/romendiratta/repos/isolated-sign-language-recognition/sample-data'

In [93]:
# Read in file that maps each sequence with ground truth label.
sample_index_mapping = pd.read_csv('index.csv').drop(columns='path')

# Read in file that maps each sign to an integer.
with open('sign_index_map.json') as f: 
    sign_index_mapping = json.load(f)

In [98]:
list(sign_index_mapping.items())[:5]

[('TV', 0), ('after', 1), ('airplane', 2), ('all', 3), ('alligator', 4)]

In [6]:
index.head()

,participant_id,sequence_id,sign
0,26734,1000035562,blow
1,28656,1000106739,wait
2,16069,100015657,cloud
3,25571,1000210073,bird
4,62590,1000240708,owie


In [8]:
# Split index into train and test set. Use stratify to ensure equal sampling across classes.
X_train, X_test, y_train, y_test = train_test_split(index[['participant_id','sequence_id']], index['sign'], 
                                                    test_size=0.2, stratify=index['sign'])

# Combine data and label into sigle dataframe.
train = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
test = pd.concat([X_test, y_test], axis=1).reset_index(drop=True)

In [102]:
# Total number of frames that will be provided to the model. (Transformer Block Size)
MAX_FRAMES_LENGTH = 3

def get_train_sample(sample_index_mapping, sign_mapping):
    # Iterate over every mapping containg samples.
    for index in sample_index_mapping.index:
        # Get the metadata associated with the sample.
        metadata = sample_index_mapping.loc[index]
        # Get all files containing sample data.
        files = glob.glob(f"{BASE_PATH}/participant_id={metadata['participant_id']}/sequence_id={metadata['sequence_id']}/*.csv")
        
        # Create single DataFrame containing all frames from sample.
        original_frames = pd.concat((pd.read_csv(f) for f in files), ignore_index=True) \
        .sort_values('frame') \
        .drop(columns='frame') \
        .reset_index(drop=True) 
        
        # Pad if less frames than input size.
        if len(original_frames) < MAX_FRAMES_LENGTH:
            input_frames = pd.DataFrame(0., index=np.arange(MAX_FRAMES_LENGTH), columns=original_frames.columns)
            ids = np.arange(len(original_frames))
            input_frames.loc[ids] = original_frames.values
        # Slice if more frames than input size.
        elif len(original_frames) > MAX_FRAMES_LENGTH:
            input_frames = original_frame.iloc[:MAX_FRAME_LEN]
        else:
            input_frames = original_frames
        
        # Return array of frames and assoicated sign. 
        yield(input_frames.values.astype(np.float32), sign_index_mapping[metadata['sign']])
                

In [103]:
# Ensure that generator works on small sample of data. 

test_sample_index_mapping = sample_index_mapping[(sample_index_mapping['participant_id']==16069) & \
                                                 ((sample_index_mapping['sequence_id']==1004211348) | \
                                                 (sample_index_mapping['sequence_id']==1005492440))]

for i in get_train_sample(test_sample_index_mapping, sign_index_mapping):
    print(i)

(array([[0.41268227, 0.42548835, 0.41106242, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32), 32)
(array([[0.39656773, 0.40152785, 0.39809203, ..., 0.        , 0.        ,
        0.        ],
       [0.39905816, 0.40365937, 0.40102798, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32), 159)


In [107]:
# Create TensorFlow Dataset for model input. 
train_dataset = tf.data.Dataset.from_generator(
    generator=lambda: get_train_sample(test_sample_index_mapping, sign_index_mapping),
    output_signature=(
        tf.TensorSpec(shape=(MAX_FRAMES_LENGTH, 1086), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.uint8)
    )
)

In [108]:
for data, label in train_dataset.batch(2).take(1):
    print(data)
    print(label)

tf.Tensor(
[[[0.41268227 0.42548835 0.41106242 ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]

 [[0.39656773 0.40152785 0.39809203 ... 0.         0.         0.        ]
  [0.39905816 0.40365937 0.40102798 ... 0.         0.         0.        ]
  [0.         0.         0.         ... 0.         0.         0.        ]]], shape=(2, 3, 1086), dtype=float32)
tf.Tensor([ 32 159], shape=(2,), dtype=uint8)
